In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_filer_date = dbutils.widgets.get("p_file_date")


#### Produce Driver Standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Find the Race  year for which  the data is reprocessed 

In [0]:
# race_result_list = spark.read.parquet(f"{presentation_folder_path}/race_results") \
#     .filter(f"file_date = '{v_filer_date}'") \
#     .select("race_year") \
#     .distinct() \
#     .collect()   

race_result_df = spark.read.parquet(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_filer_date}'") 

In [0]:
# race_result_list
race_year_list = df_column_to_list(race_result_df,"race_year")

In [0]:
# race_year_list = []
# for race_year in race_result_list:
#     race_year_list.append(race_year.race_year)
# print(race_year_list)


In [0]:
from pyspark.sql.functions import col

In [0]:
race_result_df = spark.read.parquet(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))


In [0]:
# display(race_result_df)

In [0]:
from pyspark.sql.functions import sum, count, when, col

driver_standings_df = race_result_df \
    .groupBy("race_year","driver_name","driver_nationality","team") \
    .agg(sum("points").alias("total_points"),count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(driver_standings_df.filter("race_year == 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))

final_df = driver_standings_df.withColumn("rank",rank().over(driver_rank_spec))

In [0]:
display(final_df.filter("race_year == 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins,rank


In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings")

# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")

overwrite_function(final_df, 'f1_presentation', 'driver_standings', 'race_year')


In [0]:
%sql

select * from f1_presentation.driver_standings;

driver_name,driver_nationality,team,total_points,wins,rank,race_year
Alberto Ascari,Italian,Ferrari,53.5,6,1,1952
Nino Farina,Italian,Ferrari,27.0,0,2,1952
Piero Taruffi,Italian,Ferrari,22.0,1,3,1952
Rudi Fischer,Swiss,Ferrari,10.0,0,4,1952
Mike Hawthorn,British,Cooper,10.0,0,4,1952
Robert Manzon,French,Gordini,9.0,0,6,1952
Troy Ruttman,American,Kuzma,8.0,1,7,1952
Luigi Villoresi,Italian,Ferrari,8.0,0,8,1952
José Froilán González,Argentine,Maserati,6.5,0,9,1952
Jean Behra,French,Gordini,6.0,0,10,1952


In [0]:
%sql
-- drop table  f1_presentation.driver_standings;